In [5]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
data = pd.read_pickle('D:/PSED/Jarvis_cfid/jarvis_cfid.pkl')
data_w_modulus = data.copy()

data_w_modulus.drop(data_w_modulus[data_w_modulus['bulk_modulus_kv'] == 'na'].index, inplace=True)
data_w_modulus.drop(data_w_modulus[data_w_modulus['shear_modulus_gv'] == 'na'].index, inplace=True)
data_w_modulus.drop(data_w_modulus[data_w_modulus['bulk_modulus_kv'] <= 0].index, inplace=True)
data_w_modulus.drop(data_w_modulus[data_w_modulus['shear_modulus_gv'] <= 0].index, inplace=True)

In [7]:
data_downselect = data_w_modulus.copy()
for idx in data_downselect.index:
    if not set(data_downselect.atoms[idx]['elements']).isdisjoint({'H', 'F', 'Cl', 'Br', 'I', 'At', 'He', 'Ne', 'Ar', 'Kr', 'Xe', 'Rn',
    'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Tc',
    'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr'}):
        data_downselect.drop(idx, inplace=True)

In [8]:
data_downselect.shape

(10898, 64)

In [3]:
eles = set()
for idx in data_downselect.index:
    eles = eles.union(set(data_downselect.atoms[idx]['elements'])) 

In [5]:
print(list(eles))

['Rb', 'V', 'Ru', 'S', 'K', 'B', 'Cd', 'Nb', 'Hf', 'Pb', 'Sr', 'Ga', 'Cr', 'Rh', 'Ir', 'Co', 'Ni', 'Mg', 'Pd', 'Se', 'Pt', 'Mo', 'Y', 'Hg', 'Ti', 'O', 'Cu', 'In', 'Na', 'Te', 'Ta', 'Zr', 'Tl', 'N', 'Li', 'Zn', 'Fe', 'Sc', 'Re', 'As', 'P', 'Be', 'W', 'Si', 'Ba', 'Ge', 'Ca', 'Ag', 'Os', 'Au', 'Al', 'Cs', 'Sn', 'Mn', 'Bi', 'C', 'Sb']


In [9]:
data_downselect.to_pickle('D:/PSED/Jarvis_cfid/data_downselect.pkl')

In [13]:
np.sum((data_downselect['crys'] == 'tetragonal') & (data_downselect['formation_energy_peratom'] > 0))

1060

In [7]:
cgcnn_features = pd.read_pickle('D:/PSED/Jarvis_cfid/cgcnn_features_sorted.pkl')
cgcnn_features.sort_index()
cgcnn_features_down = cgcnn_features.loc[data_downselect.index]

## Read Jarvis CFID data

In [ ]:
import os, json
import pandas as pd
import numpy as np
import pickle

f = open(os.path.join('D:\\', 'PSED', 'Jarvis_cfid', 'd3-5-16-2021.json'), 'r')
data3d = json.load(f)
f.close()

df = pd.DataFrame(data3d)

df.to_pickle('D:/PSED/jarvis_cfid.pkl')

Super hard family. Discarded.

In [24]:
superhard = data_w_modulus.copy()
for idx in superhard.index:
    if not set(superhard.atoms[idx]['elements']).issubset({'B', 'C', 'N', 'O', 'Si', 'Cr', 'Nb', 'Mo', 'Ta', 'W', 'Re'}):  # Cr, 'Al', 'Ti', 
        superhard.drop(idx, inplace=True)

In [2]:
superhard = pd.read_pickle('D:/PSED/Jarvis_cfid/superhard.pkl')

## CGCNN feature extraction.

Prepare `id_prop.csv` and `cif` files for CGCNN

In [ ]:
id_prop = pd.DataFrame(index=data_w_modulus.index)
id_prop['props'] = data_w_modulus.formation_energy_peratom
id_prop.to_csv('id_prop.csv')

from jarvis.core.atoms import Atoms
for ind in data_w_modulus.index:
    Atoms.from_dict(data_w_modulus.atoms[ind]).write_cif('D:/PSED/jarvis_cifs/'+str(ind)+'.cif')

Get the graph embeddings of materials with modulus

In [19]:
import pandas as pd
cgcnn_features = pd.read_pickle('D:/PSED/Jarvis_cfid/cgcnn_features.pkl')
cgcnn_features.set_index('cif_id', inplace=True)
cgcnn_features.index = cgcnn_features.index.astype('int')
cgcnn_features.index.name = None

cgcnn_features = cgcnn_features.sort_index()
cgcnn_features.to_pickle('cgcnn_features_sorted.pkl')